In [ ]:
!pip install langchain_community
!pip install langchain_openai
!pip install wikipedia
!pip install faiss-cpu
!pip install openai

In [ ]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

api_key=" " # write down your real api key

os.environ["OPENAI_API_KEY"] = api_key
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [ ]:
def build_example_text():
    return """
Original:
Once upon a time, at the far end of a village in the countryside, a great oak tree stood beside the wheat fields.
Villagers whispered that after sunset, strange sounds came from there—sometimes whispers, sometimes muffled laughter.
The elders said that the spirit of a man who once wronged the village was trapped in the tree.

One evening, two friends, Robert and Simon, were walking along the cobbled road.
Robert laughed and said, “Do you really believe in ghost stories?”
Simon replied, “When the elders say so, there must be some truth.”
Robert teased him and walked straight toward the oak tree, while Simon hesitated at a distance.

As Robert stood under the tree, a sudden gust of wind blew, and the dry oak leaves rustled.
Then came a whisper behind him—yet no one was there.
Suddenly, a shadow dropped from the branches: glowing red eyes, tangled hair, and a pale figure.
The ghost warned, “Leave this place, or you too will be bound to this tree forever!”

Terrified, the boys ran back to the village.
The townsfolk gathered, and the oldest man, Father Caspar, told them that long ago a cruel baron named Gregory had been tied to that oak and killed for his misdeeds.
Since then, his spirit had haunted the place.

The next day, the boys went to the village priest.
At midnight, the villagers gathered as the priest recited holy verses and sprinkled holy water around the oak tree.
The ground shook, the air grew cold, and then suddenly all was still.
The priest declared, “The spirit has been released.”
From that night onward, no strange sound was ever heard beneath the oak tree again.


*1. Culturally Specific Items (CSI)*

- oak tree (Western sacred/haunted tree motif)
- wheat fields (European farmland)
- cobbled road (typical European village imagery)
- names Robert and Simon (Western personal names)
- Father Caspar (village priest) (Christian religious authority)
- baron Gregory (feudal landlord figure)
- holy verses & holy water (Christian ritual for exorcism)



*2. Replacements (Bengal Localized)*

- oak tree → তালগাছ (talgach, palm tree, common in Bengali folklore)
- wheat fields → ধানের ক্ষেত (dhaner khet, rice field)
- cobbled road → গ্রামের মাটির পথ (gramer matir poth, village mud road)
- Robert → রবি (Rabi), Simon → সুমন (Suman)
- Father Caspar (priest) → কাসেম দাদু (Kasem dadu, wise village elder)
- baron Gregory → জমিদার গজেন্দ্র (zamindar Gajendra, oppressive landlord)
- holy verses & holy water → তান্ত্রিকের মন্ত্র (tantriker mantra, tantric ritual with chants)


*3. Result*

গায়ের শেষ প্রান্তে ধানের ক্ষেতের পাশেই
একটা উঁচু তাল গাছ দাঁড়িয়ে আছে বহু বছর
ধরে লোকমুখে শোনা যায় সন্ধ্যার পর কেউ
ওখান দিয়ে গেলে অদ্ভুত শব্দ শোনা যায়
কখনো ফিসফিসানি কখনো আবার কারো চাপা হাসির
শব্দ গ্রামের বয়স্করা বলে ওখানে এক
তালগাছের ভূত থাকে একদিন সন্ধ্যায় রবি আর
সুমন গ্রামের রাস্তা ধরে হাঁটছিল তারা ছিল
দুই বন্ধু ভয় দেওয়ার তেমন একটা ছিল না
তাদের আচ্ছা সুমন তুই কি সত্যিই বিশ্বাস
করিস এইসব ভূতের গল্পে আরে বাবা গ্রামের
বড়রা যখন বলে কিছু না কিছু তো সত্যিই আছে
বল ধুর ভূত টুত কিছু নেই তুই যদি সাহসী হস
চল ওই তাল গাছের নিচে একটু বসে যাই ধ্যাত
ওটা নিয়ে খেলতে নেই রে রবি তোর সাহস বেশি
তুই নিজেই যা আমি বাপু ওখানে যাব না রবি
হাসতে হাসতে এগিয়ে গেল তালগাছের দিকে
সুমন একটু দূর থেকে দাঁড়িয়ে দেখছিল হঠাৎ
করে বাতাসের একটা প্রবল ঝাপটা এলো
তালগাছের শুকনো পাতাগুলো হাওয়ায় দুলতে
লাগলো রবি তালগাছের নিচে দাঁড়িয়ে পড়লো
হঠাৎই পেছন থেকে একটা ফিসফিসানি শুনতে পেল
রবি দ্রুত চারপাশে তাকালো কিন্তু কাউকে
দেখতে পেল না সুমন ভয়ে কাঁপতে কাঁপতে
চেচিয়ে বলল
রবি কে কোথা বলল রে ওটা আমি জানিনা রে
হঠাৎ করেই তালগাছের ডালের ওপর থেকে একটা
কালো ছায়া নেমে এলো তারা কিছু বোঝার আগেই
সেই ছায়াটা গাছের উড়ির দিকে মিশে গেল
কেন এসেছো
এখানে কে আছো কে আছো ওখানে
সাম সামনে আসো কথাটা বলার সঙ্গে সঙ্গে তাল
গাছের গায়ে একটা ম্লান ভৌতিক চেহারা ফুটে
উঠলো তার লাল চোখ লম্বা চুল আর কম্পালসার
শরীর দেখে দুই বন্ধু গলা শিখিয়ে দিলেন কেন
এসেছো এখানে
তোমরা কেন
এসেছো তোমরা তোমরা তো শুধু চলে যাও
নয়তো আমার মত তোমার
এই ডাল গাছে বন্দি হয়ে থাকতে হবে
রবি আর সুমন এক মুহূর্ত দেরি না করে পেছন
ফিরে ছুটতে লাগালো রবি আর সুমন ততক্ষণে
গ্রামের ভেতরে ঢুকে পড়েছে তারা একদম
নিঃশ্বাস নিতে পারছিল না গ্রামের লোকজন
তাদের দেখে ছুটে এলো কি হয়েছে রে তোদের
কেন এভাবে ছুটে ছুটে আসছিস তালগাছের ভূত
ভূত দেখেছি ভূত দেখেছি বোন কাকা ভূত
সত্যিই আছে আমরা ওর কথা শুনেছি গ্রামের
লোকজন হতবাক হয়ে গেল কাসেম দাদু যিনি
গ্রামের সবচেয়ে বৃদ্ধ মানুষ ধীর পায়ে
এসে
বললেন আমি অনেকদিন আগেই
বলেছিলাম ওখানে যেও না ওখানে এক হাত
আছে তাও আবার বন্দি আত্মা কার আত্মা দাদু
কার বহু বছর
আগে এক টাকার
ছিল নাম ছিল
গজেন্দ্র সেই গ্রামের মানুষের
ওপর অনেক অত্যাচার
করেছিল একদিন গ্রামের লোকজন তাকে ধরে ওই
তাল গাছে সঙ্গে পেঁথে রাখে এবং মেরে ফেলে
সেই থেকেই ওরা ওই তালগাছে বন্দি হয়ে
আছে সেদিন রাতে গ্রামের পঞ্চায়েতও
একত্রিত হলো সবাই মিলে সিদ্ধান্ত নিল
তালগাছের ওখানে আর কেউ যাবে না কিন্তু রবি
আর সুমন ঠিক করল তারা এক সাধু বাবার কাছে
যাবে পরদিন তারা এক তান্ত্রিকের কাছে
গিয়ে পুরো ঘটনা বলল তিনি বললেন ওর
মুক্তির ব্যাপার ব্যবস্থা করতে হবে না হলে
আরো বিপদ হবে তিনি কিছু মন্ত্র পড়লেন এবং
বললেন যে রাত বারোটায় তালগাছের নিচে যেতে
হবে রাত বারোটায় সবাই মিলে তালগাছের কাছে
গেল তান্ত্রিক বিশেষ মন্ত্র পড়তে লাগলেন
হঠাৎ তালগাছ প্রচন্ড কাঁপতে লাগলো বাতাস
তীব্র হয়ে উঠলো তারপর হঠাৎ সব শান্ত হয়ে
গেল তান্ত্রিক বললেন আত্মা মুক্তি পেয়েছে
পরের দিন থেকে তালগাছের নিচে আর কোন
অদ্ভুত আওয়াজ শোনা যায়নি

"""

def build_cot_ce_se_prompt(text, cultural_context=""):
    example_hint = build_example_text()

    example_char_names = [
        "আমীর", "উজীর", "কাজী", "খালিম", "গরীব চাষী", "চন্দরা", "ছিদাম", "জেলে",
        "জয়াড়ী", "ডাইন", "ডাকাত", "তাঁতী", "তীরন্দাজ", "দুখিরাম", "দয়াল",
        "বুখারা", "ভালক", "মাংসওয়ালা", "মাদ্রাসা", "মেয়েটি", "মোল্লাসাহেব", "যুবর্তী",
        "রফি", "রাধা", "রামলোচন", "শাহনশাহ", "সণরী", "হাফিজ", "হামদাম", 'অপর্ণা', 'ঈশ্বর',
        'কুমির', 'খুঁড়ো', 'খোরু', 'গিন্নি', 'গুরুদেব',
        'জমিদার বাবু', 'দাদা', 'পঞ্চমন্তু', 'পেতনি', 'বউ', 'বলি', 'বাবলু',
        'বাবলুর বউ', 'মেছো পেতনি', 'রমা', 'রমেশ', 'রাজু', 'লক্ষ্মী',
        'শিয়াল', 'শ্যামল', 'সোনা', 'হরিপদ'
    ]

    name_reference_note = f"""
Names in West Bengal folktales are usually role-based, virtue-based, or relational.
Reference list: {', '.join(example_char_names)}.
"""

    return f"""
Translate the following English text into **BENGALI** in **traditional WEST BENGAL folktale style**.

1. **CSI Identification**
   - Identify all culturally specific items (CSI) in the input text that are inappropriate in  WEST BENGAL folktales.
2. **Cultural Adaptation**
   - Replace each CSI with a culturally appropriate ** WEST BENGAL traditional equivalent**.
   - Use the following Wikipedia-derived cultural context to INCREASE CULTURAL APPROPRIATENESS of your generation.
     Ground concrete details (clothing, food, region festivals, Religion, social customs) the following **Wikipedia-derived cultural context**:
     {cultural_context}
   - For names and roles, refer to the following ** WEST BENGAL folktale name reference list**:
     {name_reference_note}
3. **Folktale-style Generation**
   - Following the pattern in this **example reasoning**:
     {example_hint}
   - Produce the final story in WEST BENGAL folktale style, concise, rhythmic, and oral.
   - Use only past cultural and narrative elements, with no modern intrusions.
**Input Text:**
{text}
""".strip()



def convert_folktale_cot_ce_se(text, cultural_context=""):
    prompt = build_cot_ce_se_prompt(text, cultural_context=cultural_context)

    response = client.chat.completions.create(
         model="gpt-5",
        messages=[
            {"role": "system", "content": "You are an AI storyteller who transforms global fairy tales into Korean traditional folktale style, considering cultural transfer with CEreasoning. Do not generate sentences that are too long. Keep sentences concise and readable."},
            {"role": "user", "content": prompt}
        ],
        )
    return response.choices[0].message.content

from langchain_community.document_loaders import WikipediaLoader

topics = [
    "Culture of West Bengal",
    "Religion in West Bengal",
    "Bengali cuisine",
    "Folktale of India",
    "Bengali Muslim wedding",
    "Bengali Hindu wedding",
    "Bangladeshi folk literature",
    "Architecture of Bengal",
    "Thakurmar Jhuli"
]

docs = []
for t in topics:
    docs.extend(WikipediaLoader(query=t, lang="en").load())

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", api_key=os.environ["OPENAI_API_KEY"])
db = FAISS.from_documents(chunks, embeddings)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 8})

if __name__ == "__main__":
    english_text_1 =  """
A long time ago, a child was born to a queen and king and she was called Snow White.  When the queen died, the king married again.  This new queen was wicked and hated Snow white.  The queen gave orders that Snow White was to be treated as a servant.
Snow White grew very beautiful and one day a Prince riding by, saw her at work and fell in love with her.
The queen was beautiful too, and every day she asked her Magic Mirror, "Who is the fairest in the land?" and the mirror always answered, "You are the fairest one of all".
But one day the mirror answered Snow White was the fairest in the land, and in a rage the queen gave orders to one of her Huntsmen to take Snow White into the woods and kill her.
The Huntsman had a kind heart and couldn't do the deed so told her to run away.  She fled into the woods where Seven little dwarfs lived.  Their house was small and strange.

 Snow White entered the little house and finding it very untidy, started to clean up.  Upstairs she found seven little beds.  She was very tired and stretching out on one of the beds, was soon asleep.
When the Dwarfs came home they were surprised to find Snow White and after some argument, decided to let her stay.  She promised to cook and look after them.
The Queen discovered where Snow White was living and disguising herself as a witch, took a poisoned apple and set out for the Dwarfs cottage.  She gave Snow White the poisoned apple to eat and as soon as she bit the apple, she sank into unconsciousness.
Thinking she was dead, the Dwarfs built a glass coffin and put her in it. For days she lay in the forest in her glass coffin.  One day, the Prince was riding through the forest looking for Snow White and found her.  He leaned over and kissed her.  She opened her eyes and sat up with a smile. Everyone was happy.  The Prince took Snow White to his palace where they were married and lived happily ever after.

"""

    query = "Bengali clothing cuisine weddings festivals religion"
    results = retriever.invoke(query)
    for i, r in enumerate(results, 1):
        print(f"Chunk {i} (metadata: {r.metadata}):\n{r.page_content[:300]}...\n")

    cultural_context = "\n".join([r.page_content for r in results])
    output = convert_folktale_cot_ce_se(english_text_1, cultural_context)

    print("================================================================================")
    print("Final Folktale Output")

    print(output)
